## 5.1 决策树
### 5.1.1 决策树模型
内部节点、叶节点
### 5.1.2 决策树与if - then 规则
可读性来源<br>
重要性质：互斥并且完备
### 5.1.3 决策树和条件概率分布
看三张图
### 5.1.4 决策树学习
与训练集矛盾较小 + 具有很好的泛化能力<br>
决策树的生成对应于模型的局部选择，决策树的剪枝对应于模型的全局选择。

## 5.2 特征选择
### 5.2.1 特征选择问题
### 5.2.2 信息增益
在信息论与概率统计中，熵是表示随机变量不确定性的度量。<br>
信息增益表示得知特征X的信息而使得类Y的信息的不确定性减少的程度。信息增益大的特征具有更强的分类能力。<br>
### 5.2.3 信息增益比
解决偏向于选择取值较多的特征的问题

## 5.3 决策树的生成
### 5.3.1 ID3算法
利用信息增益<br>
ID3算法只有树的生成，所以该算法生成的树容易产生过拟合
### 5.3.2 C4.5的生成算法
利用信息增益比进行特征的选择

## 5.4 决策树的剪枝
(防止过拟合往往就是在控制模型的复杂度)<br>
决策树的剪枝往往通过极小化决策树整体的损失函数或代价函数来实现.<br>
决策树生成学习局部的模型，而决策树剪枝学习整体的模型。

## 5.5 CART 算法
### 5.5.1 CART 生成
决策树的生成就是递归地构建二叉决策树的过程。对回归树用平方误差最小化准则，对分类树用基尼指数最小化准则，进行特征选择，生成二叉树。<br>
在各个特征中选最优，在优中选优
### 5.5.2 CART 剪枝
参考链接：https://zhuanlan.zhihu.com/p/76709712

### 习题5.1

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log
import pprint

In [3]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [4]:
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)

In [7]:
# 熵
def calc_ent(datasets):
    data_length = len(datasets)
    label_count = {}
    for i in range(data_length):
        label = datasets[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    ent = -sum([(p / data_length) * log(p / data_length, 2)
                for p in label_count.values()])
    return ent

# 经验条件熵
def cond_ent(datasets, axis=0):
    data_length = len(datasets)
    feature_sets = {}
    for i in range(data_length):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_ent = sum(
        [(len(p) / data_length) * calc_ent(p) for p in feature_sets.values()])
    return cond_ent

# 信息增益
def info_gain(ent, cond_ent):
    return ent - cond_ent

def info_gain_train(datasets):
    count = len(datasets[0]) - 1
    ent = calc_ent(datasets)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain(ent, cond_ent(datasets, axis=c))
        best_feature.append((c, c_info_gain))
        print('特征({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))
    # 比较大小
    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益最大，选择为根节点特征'.format(labels[best_[0]])

In [8]:
info_gain_train(np.array(datasets))

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.324
特征(有自己的房子) - info_gain - 0.420
特征(信贷情况) - info_gain - 0.363


'特征(有自己的房子)的信息增益最大，选择为根节点特征'